# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../../")

from Clust.app import data_manipulation, data_preprocessing
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

##### #0. Common###########################################################################################
clean_level_list =[0, 4]

===integrationStart===
===integrationEnd===
========== create bucket ==========
========== write success ==========
========== Data Save Success ==========
{'data_name': 'regression_energy_trainX_cleanLevel0', 'ingestion_param': {'start_time': '2016-01-11', 'end_time': '2016-04-15', 'ms_list_info': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 600}, 'clean_level': 0, 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}, '_id': ObjectId('643f4dc20dbacffcc28fe9c2')}
===integrationStart===
===integrationEnd===
========== write success =====

=====  RH_1 - 25%: 37.433333333333294 75% 42.363333333333294
===========IQR Low~High: -455.5666666666667 ~ 535.3633333333333
=====  RH_2 - 25%: 38.09 75% 43.296666666666695
===========IQR Low~High: -482.5766666666692 ~ 563.963333333336
=====  RH_3 - 25%: 37.06 75% 40.328333333333354
===========IQR Low~High: -289.7733333333352 ~ 367.16166666666857
=====  RH_4 - 25%: 35.33166666666665 75% 40.5
===========IQR Low~High: -481.5016666666684 ~ 557.3333333333351
=====  RH_5 - 25%: 44.9666666666667 75% 53.82750000000002
===========IQR Low~High: -841.1166666666658 ~ 939.9108333333324
=====  RH_6 - 25%: 29.12583333333335 75% 81.53
===========IQR Low~High: -5211.290833333332 ~ 5321.946666666665
=====  RH_7 - 25%: 30.89 75% 37.3266666666667
===========IQR Low~High: -612.7766666666703 ~ 680.993333333337
=====  RH_8 - 25%: 38.45571428571428 75% 44.56513888888885
===========IQR Low~High: -572.4867460317431 ~ 655.5075992063463
=====  RH_9 - 25%: 38.23 75% 43.09
===========IQR Low~High: -447.77000000000

In [ ]:
##### #1. For Forecasting##################################################################################################################################   
data_name_list=['air_유치원1']
integration_freq_sec = 60 * 5
data_mode_list=["train", "test"]
for clean_level in clean_level_list:
    for data_name in data_name_list:
        for data_mode in data_mode_list:
            process_param = data_preprocessing.get_process_param_by_level(clean_level)
            ingestion_param={}
            integration_param = {
                    #"integration_frequency":integration_freq_sec,
                        "param":{},
                        "method":"meta",
                        "integration_duration":"common"
            }
            integration_param['integration_frequency'] = integration_freq_sec
            if data_name==data_name_list[0]: # Hs1SwineFarmWithWeatherTime
                if data_mode == 'train':
                    ingestion_param['start_time'] = "2021-02-01 00:00:00"
                    ingestion_param['end_time']  ="2021-03-10 00:00:00"
                elif data_mode =='test':
                    ingestion_param['start_time']  ="2021-03-10 00:00:00"
                    ingestion_param['end_time']  ="2021-03-17 00:00:00"
                ingestion_param['ms_list_info'] = [['farm_swine_air', 'HS2'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]

            from Clust.clust.data import data_interface
            multiple_dataset = data_interface.get_data_result("multiple_ms_by_time", db_client, ingestion_param)

            from Clust.clust.preprocessing import processing_interface 
            multiple_dataset = processing_interface.get_data_result('step_3', multiple_dataset, process_param)

            from Clust.clust.integration.integrationInterface import IntegrationInterface
            data = IntegrationInterface().multipleDatasetsIntegration(integration_param, multiple_dataset)
            
            # save to influxdb
#             data_manipulation.ingestion_processing_integration(db_client, ingestion_param, process_param, integration_param)
            ms_name = "forecasting_"+ data_name+'_' + data_mode+ '_cleanLevel' + str(clean_level)
            db_client.write_db(bucket_name, ms_name, data)
            
            # save to mongodb
            meta_info={"data_name": ms_name,"ingestion_param": ingestion_param, 
                       "integration_param":integration_param, "clean_level":clean_level, "process_param":process_param}
            collection_name = "forecasting_"+ data_name
            mongo_client.insert_document(bucket_name, collection_name, meta_info)
            print(meta_info)          

In [2]:

##### #1. For Forecasting##################################################################################################################################   
data_name_list=['Hs2SwineFarmWithWeatherTime', 'gunwiStrawberryWeather', 'strawberryOpen']
integration_freq_sec = 60 * 5
data_mode_list=["train", "test"]
for clean_level in clean_level_list:
    for data_name in data_name_list:
        for data_mode in data_mode_list:
            process_param = data_preprocessing.get_process_param_by_level(clean_level)
            ingestion_param={}
            integration_param = {
                    #"integration_frequency":integration_freq_sec,
                        "param":{},
                        "method":"meta",
                        "integration_duration":"common"
            }
            integration_param['integration_frequency'] = integration_freq_sec
            if data_name==data_name_list[0]: # Hs1SwineFarmWithWeatherTime
                if data_mode == 'train':
                    ingestion_param['start_time'] = "2021-02-01 00:00:00"
                    ingestion_param['end_time']  ="2021-03-10 00:00:00"
                elif data_mode =='test':
                    ingestion_param['start_time']  ="2021-03-10 00:00:00"
                    ingestion_param['end_time']  ="2021-03-17 00:00:00"
                ingestion_param['ms_list_info'] = [['farm_swine_air', 'HS2'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]

            elif data_name==data_name_list[1]: # gunwiStrawberryWeather
                    if data_mode == 'train':
                        ingestion_param['start_time'] = "2022-01-22 00:00:00"
                        ingestion_param['end_time']  ="2022-02-25 00:00:00"
                    elif data_mode =='test':
                        ingestion_param['start_time']  ="2022-02-25 00:00:00"
                        ingestion_param['end_time']  ="2022-02-28 00:00:00"
                    ingestion_param['ms_list_info'] = [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]
                    

            elif data_name ==data_name_list[2]: # strawberryOpen
                if data_mode == 'train':
                        ingestion_param['start_time'] = "2022-01-22 00:00:00"
                        ingestion_param['end_time']  ="2022-02-25 00:00:00"
                elif data_mode =='test':
                    ingestion_param['start_time']  ="2022-02-25 00:00:00"
                    ingestion_param['end_time']  ="2022-02-28 00:00:00"
                ingestion_param['ms_list_info'] = [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]
                
            from Clust.clust.data import data_interface
            multiple_dataset = data_interface.get_data_result("multiple_ms_by_time", db_client, ingestion_param)

            from Clust.clust.preprocessing import processing_interface 
            multiple_dataset = processing_interface.get_data_result('step_3', multiple_dataset, process_param)

            from Clust.clust.integration.integrationInterface import IntegrationInterface
            data = IntegrationInterface().multipleDatasetsIntegration(integration_param, multiple_dataset)
            
            # save to influxdb
#             data_manipulation.ingestion_processing_integration(db_client, ingestion_param, process_param, integration_param)
            ms_name = "forecasting_"+ data_name+'_' + data_mode+ '_cleanLevel' + str(clean_level)
            db_client.write_db(bucket_name, ms_name, data)
            
            # save to mongodb
            meta_info={"data_name": ms_name,"ingestion_param": ingestion_param, 
                       "integration_param":integration_param, "clean_level":clean_level, "process_param":process_param}
            collection_name = "forecasting_"+ data_name
            mongo_client.insert_document(bucket_name, collection_name, meta_info)
            print(meta_info)          

===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Save Success ==========
{'data_name': 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0', 'ingestion_param': {'start_time': '2021-02-01 00:00:00', 'end_time': '2021-03-10 00:00:00', 'ms_list_info': [['farm_swine_air', 'HS2'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 300}, 'clean_level': 0, 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}, '_id': ObjectId('643f4ddd0dbacffcc28fe9ca')}
===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Save Success ==========
{'data_

=====  CO2ppm - 25%: 1717.0 75% 3642.25
===========IQR Low~High: -190808.0 ~ 196167.25
=====  H2Sppm - 25%: 3.0 75% 5.0
===========IQR Low~High: -197.0 ~ 205.0
=====  Humidity - 25%: 59.2 75% 64.7
===========IQR Low~High: -490.8 ~ 614.7
=====  NH3ppm - 25%: 131.0 75% 32767.0
===========IQR Low~High: -3263469.0 ~ 3296367.0
=====  Temperature - 25%: 23.7 75% 26.1
===========IQR Low~High: -216.30000000000024 ~ 266.10000000000025
=====  out_humid - 25%: 45.0 75% 90.0
===========IQR Low~High: -4455.0 ~ 4590.0
=====  out_pressure - 25%: 1007.3 75% 1012.0
===========IQR Low~High: 537.2999999999954 ~ 1482.0000000000045
=====  out_rainfall - 25%: 0.0 75% 0.0
===========IQR Low~High: 0.0 ~ 0.0
=====  out_sunshine - 25%: 0.0 75% 0.0
===========IQR Low~High: 0.0 ~ 0.0
=====  out_temp - 25%: 5.074999999999999 75% 14.25
===========IQR Low~High: -912.4250000000001 ~ 931.7500000000001
=====  out_wind_direction - 25%: 70.0 75% 270.0
===========IQR Low~High: -19930.0 ~ 20270.0
=====  out_wind_speed - 25

========== Data Save Success ==========
{'data_name': 'forecasting_gunwiStrawberryWeather_test_cleanLevel4', 'ingestion_param': {'start_time': '2022-02-25 00:00:00', 'end_time': '2022-02-28 00:00:00', 'ms_list_info': [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 300}, 'clean_level': 4, 'process_param': {'refine_param': {'removeDuplication': {'flag': True}, 'staticFrequency': {'flag': True, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': True, 'param': {'outlierDetectorConfig': [{'algorithm': 'IQR', 'percentile': 99, 'alg_parameter': {'weight': 100}}]}}}, 'imputation_param': {'flag': False, 'imputation_method': [{'min': 0, 'max': 2, 'method': 'linear', 'parameter': {}}], 'totalNonNanRatio': 90}}, '_id': ObjectId('643

========== Data Save Success ==========
{'data_name': 'forecasting_strawberryOpen_test_cleanLevel4', 'ingestion_param': {'start_time': '2022-02-25 00:00:00', 'end_time': '2022-02-28 00:00:00', 'ms_list_info': [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 300}, 'clean_level': 4, 'process_param': {'refine_param': {'removeDuplication': {'flag': True}, 'staticFrequency': {'flag': True, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': True, 'param': {'outlierDetectorConfig': [{'algorithm': 'IQR', 'percentile': 99, 'alg_parameter': {'weight': 100}}]}}}, 'imputation_param': {'flag': False, 'imputation_method': [{'min': 0, 'max': 2, 'method': 'linear', 'parameter': {}}], 'totalNonNanRatio': 90}}, '_id': ObjectId('643f4e0d0db

In [ ]:
##### #2. For Regression######################################################################################################################################
data_mode_list =['trainX', 'trainy', 'testX', 'testy']
trainStartTime = "2016-01-11"
trainEndTime = "2016-04-15"
testStartTime = "2021-01-01"
testEndTime = "2021-02-12"
bucket_name ="integration"
for clean_level in clean_level_list:
    # TODO p1 모두 없앨 것
    process_param = data_preprocessing.get_process_param_by_level(clean_level)
    for data_mode in data_mode_list:
        ingestion_param ={}
        integration_param = {
            #"integration_frequency":integration_freq_sec,
            "param":{},
            "method":"meta",
            "integration_duration":"common"
        }
        if data_mode == 'trainX':
            ingestion_param['start_time'] = trainStartTime
            ingestion_param['end_time'] = trainEndTime
            ingestion_param['ms_list_info'] = [['life_indoor_environment', 'humidityTrain_10min'], 
                        ['life_indoor_environment', 'temperatureTrain_10min'], 
                        ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']]
            integration_param["integration_frequency"] = 60 * 10 # 10분

        elif data_mode =='testX':
            ingestion_param['start_time'] = testStartTime
            ingestion_param['end_time'] = testEndTime
            ingestion_param['ms_list_info'] = [['life_indoor_environment', 'humidityTest_10min'], 
                        ['life_indoor_environment', 'temperatureTest_10min'], 
                        ['weather_outdoor_environment', 'belgiumChieverseAirportTest_10min']]
            integration_param["integration_frequency"]= 60 * 10 # 10분

        elif data_mode == 'trainy':
            ingestion_param['start_time'] = trainStartTime
            ingestion_param['end_time'] = trainEndTime
            ingestion_param['ms_list_info'] = [['life_indoor_environment', 'applianceEnergyDatasetTrainy_1day']]
            integration_param["integration_frequency"] = 60 * 60 * 24 # 24시간

        elif data_mode =='testy':
            ingestion_param['start_time'] = testStartTime
            ingestion_param['end_time'] = testEndTime
            ingestion_param['ms_list_info'] = [['life_indoor_environment', 'applianceEnergyDatasetTesty_1day']]
            integration_param["integration_frequency"] = 60 * 60 * 24 # 24시간
            
        # dataset ingestion---> data preprocessing ---> data integration ---> information save
        # 1. Ingestion multiple dataset

        # save to influxdb
        data = data_manipulation.ingestion_processing_integration(db_client, ingestion_param, process_param, integration_param)
        ms_name = "regression_energy_"+ data_mode + '_cleanLevel' + str(clean_level)
        db_client.write_db(bucket_name, ms_name, data)
        
        # save to mongodb
        meta_info={"data_name": ms_name,"ingestion_param": ingestion_param, 
                   "integration_param":integration_param, "clean_level":clean_level, "process_param":process_param}
        collection_name = "regression_energy"
        mongo_client.insert_document(bucket_name, collection_name, meta_info)
        print(meta_info)